In [2]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.3.0'

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
mlp_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
mlp_model.summary()

In [ ]:
inputs = tf.keras.Input(shape=(28, 28))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

mlp_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)

mlp_model = MLP_Model()

In [ ]:
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(x_train, y_train, epochs=5)
mlp_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
mlp_model.summary()

In [ ]:
# SavedModel 저장

tf.saved_model.save(mlp_model, "./mlp_model/")
# tf.keras.models.save_model(mlp_model, "./mlp_model/")
# mlp_model.save("./mlp_model/")

In [ ]:
# SavedModel 복원

saved_model = tf.keras.models.load_model("./mlp_model/")
saved_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
# HDF5 저장

mlp_model.save("mlp_model.h5")
# mlp_model.save("./mlp_model", save_format='h5')

In [ ]:
# HDF5 복원

h5_model = tf.keras.models.load_model("mlp_model.h5")
h5_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
# Saved Model 변환

converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
tflite_model = converter.convert()
with tf.io.gfile.GFile('mlp_saved_model.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./") / "mlp_saved_model.tflite"
tflite_model_file.write_bytes(tflite_model)

In [ ]:
# Saved Model 테스트
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

x_test_f = np.expand_dims(x_test[999], axis=0).astype(np.float32)

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(input_index, x_test_f)
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

import matplotlib.pylab as plt

plt.imshow(x_test[999])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(y_test[999]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in x_test:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == y_test[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy
evaluate_model(interpreter)

In [ ]:
# Keras Model 변환

new_model = tf.keras.models.load_model("mlp_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()
with tf.io.gfile.GFile('mlp_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./") / "mlp_keras_model.tflite"
tflite_model_file.write_bytes(tflite_model)

In [7]:
# dynamic range 양자화
converter = tf.lite.TFLiteConverter.from_saved_model("./Chapter_4/mlp_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
with tf.io.gfile.GFile('mlp_saved_model_quant.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./Chapter_4") / "mlp_saved_model_quant.tflite"
tflite_model_file.write_bytes(tflite_quant_model)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
evaluate_model(interpreter)

c:\Project\git\android_tflite\Chapter_8


OSError: SavedModel file does not exist at: ./Chapter_4/mlp_model/{saved_model.pbtxt|saved_model.pb}

In [ ]:
# float16 양자화

converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
with tf.io.gfile.GFile('mlp_saved_model_quant_float16.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./") / "mlp_saved_model_quant_float16.tflite"
tflite_model_file.write_bytes(tflite_quant_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

x_train_f = tf.cast(x_train, tf.float32)
mnist_ds = tf.data.Dataset.from_tensor_slices((x_train_f)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter.representative_dataset = representative_data_gen
tflite_quant_model = converter.convert()
with tf.io.gfile.GFile('mlp_saved_model_quant_fullint.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./") / "mlp_saved_model_quant_fullint.tflite"
tflite_model_file.write_bytes(tflite_quant_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

x_train_f = tf.cast(x_train, tf.float32)
mnist_ds = tf.data.Dataset.from_tensor_slices((x_train_f)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
with tf.io.gfile.GFile('mlp_saved_model_quant_fullintonly.tflite', 'wb') as f:
    f.write(tflite_model)

import pathlib
tflite_model_file = pathlib.Path("./") / "mlp_saved_model_quant_fullintonly.tflite"
tflite_model_file.write_bytes(tflite_quant_model)